In [1]:
from basic_fcn import *
import time
from torch.utils.data import DataLoader
import torch
import gc
import voc
import torchvision.transforms as standard_transforms
import util
import numpy as np
import sys
import math
import copy

class MaskToTensor(object):
    def __call__(self, img):
        return torch.from_numpy(np.array(img, dtype=np.int32)).long()

# Initialize Weights with Xavier Weight Initialization
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data) #xavier not applicable for biases

In [2]:
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
input_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
target_transform = MaskToTensor()

train_dataset =voc.VOC('train', transform=input_transform, target_transform=target_transform)
val_dataset = voc.VOC('val', transform=input_transform, target_transform=target_transform)
test_dataset = voc.VOC('test', transform=input_transform, target_transform=target_transform)

train_loader = DataLoader(dataset=train_dataset, batch_size= 16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 16, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size= 16, shuffle=False)

epochs = 20
n_class = 21

fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)

FCN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bnd5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (deconv1): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  (bn1): Ba

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=5e-4)

# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
criterion = nn.CrossEntropyLoss()
fcn_model = fcn_model.to(device)
max_model = fcn_model

earlystop = 3

In [4]:
def train():
    """
    Train a deep learning model using mini-batches.

    - Perform forward propagation in each epoch.
    - Compute loss and conduct backpropagation.
    - Update model weights.
    - Evaluate model on validation set for mIoU score.
    - Save model state if mIoU score improves.
    - Implement early stopping if necessary.

    Returns:
        None.
    """

    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())

            #if iter % 10 == 0:
            #    print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))

        current_miou_score = val(epoch)

        # Save current IoU if better than stored best
        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model) # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [5]:
def val(epoch):
    """
    Validate the deep learning model on a validation dataset.

    - Set model to evaluation mode. DONE
    - Disable gradient calculations. DONE
    - Iterate over validation data loader:
        - Perform forward pass to get outputs.
        - Compute loss and accumulate it.
        - Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the epoch.
    - Switch model back to training mode.

    Args:
        epoch (int): The current epoch number.

    Returns:
        tuple: Mean IoU score and mean loss for this validation epoch.
    """
    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    losses = []
    mean_iou_scores = []
    accuracy = []
    
    with torch.no_grad(): # we don't need to calculate the gradient in the validation/testing

        # Iterate through Validation Set
        for iter, (input, label) in enumerate(val_loader):
            # label = (16, 224, 224) / batch size 16 of 244*244 masks
            # output = (16, 21, 224, 224) / batch size 16 of 21 possible classes of 244*244 masks
            
            input, label = input.to(device), label.to(device) # both inputs and labels in device as model
            
            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)
    
    print(f"Validation Loss: {np.mean(losses)}")
    print(f"Validation IoU: {np.mean(mean_iou_scores)}")
    print(f"Validation Pixel Acc: {np.mean(accuracy)}")
    print("\n")

    fcn_model.train() #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

    return np.mean(mean_iou_scores)

def modelTest():
    """
    Test the deep learning model using a test dataset.

    - Load the model with the best weights.
    - Set the model to evaluation mode.
    - Iterate over the test data loader:
        - Perform forward pass and compute loss.
        - Accumulate loss, IoU scores, and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the test data.
    - Switch model back to training mode.

    Returns:
        None. Outputs average test metrics to the console.
    """

    fcn_model = copy.deepcopy(max_model) # Asssume model loaded with the best weights.
    
    fcn_model.eval()  # Put in eval mode (disables batchnorm/dropout) !

    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad():  # we don't need to calculate the gradient in the validation/testing

        # Iterate through Test Set
        for iter, (input, label) in enumerate(test_loader):

            input, label = input.to(device), label.to(device) # both inputs and labels in device as model

            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)

    print(f"Test Loss at Test: {np.mean(losses)}")
    print(f"Test IoU at Test: {np.mean(mean_iou_scores)}")
    print(f"Test Pixel acc at Test: {np.mean(accuracy)}")

    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

In [6]:
def exportModel(inputs):    
    """
    Export the output of the model for given inputs.

    - Set the model to evaluation mode.
    - Load the model with the best saved weights.
    - Perform a forward pass with the model to get output.
    - Switch model back to training mode.

    Args:
        inputs: Input data to the model.

    Returns:
        Output from the model for the given inputs.
    """

    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    saved_model_path = "Fill Path To Best Model"
    # TODO Then Load your best model using saved_model_path
    
    inputs = inputs.to(device)
    
    output_image = fcn_model(inputs)
    
    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!
    
    return output_image

# if __name__ == "__main__":

#     val(0)  # show the accuracy before training
#     train()
#     modelTest()

#     # housekeeping
#     gc.collect()
#     torch.cuda.empty_cache()

In [7]:
def train1():
    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())
            
            scheduler.step() # For cosine annealing learning rate

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))
        
        current_miou_score = val(epoch)

        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model)
            # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

## Q4.b Image Transformation

In [8]:
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 5
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-3)
#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [9]:
train1()

Finish epoch 0, time elapsed 4.20524525642395
Train Avg Loss: 2.8351216656821117
Validation Loss: 2.722186701638358
Validation IoU: 0.003082168655440335
Validation Pixel Acc: 0.02075419523278061


Finish epoch 1, time elapsed 3.083066940307617
Train Avg Loss: 2.7582521608897617
Validation Loss: 2.6302671943392073
Validation IoU: 0.011825486881233072
Validation Pixel Acc: 0.13970895661557034


Finish epoch 2, time elapsed 2.967177629470825
Train Avg Loss: 2.6701625301724388
Validation Loss: 2.4873415742601668
Validation IoU: 0.023267570670662038
Validation Pixel Acc: 0.29533773260978496


Finish epoch 3, time elapsed 3.142828941345215
Train Avg Loss: 2.5974900679928914
Validation Loss: 2.7595037732805525
Validation IoU: 0.022530341989280268
Validation Pixel Acc: 0.2577031527594297


Finish epoch 4, time elapsed 3.1711556911468506
Train Avg Loss: 2.5282540968486242
Validation Loss: 2.223456655229841
Validation IoU: 0.04006436661511203
Validation Pixel Acc: 0.5409829898756378


Finish epo

In [10]:
modelTest()

Test Loss at Test: 1.4396237475531442
Test IoU at Test: 0.05698964757311347
Test Pixel acc at Test: 0.7191921045064232


In [11]:
torch.save(fcn_model, "/home/sgaywala/private/PA2/cse151b251b-wi24-pa2-cse-151b-pa2/model_4b.pt")